In [ ]:
"""
#Used only for Dev

import simulator, agent, simulator_interface, behavior
from importlib import reload
reload(simulator)
reload(behavior)
reload(simulator_interface)
reload(agent)

"""

# Preparing the simulator

In [1]:
from simulator_interface import open_session, close_session
import random as rd
import math
agents = [
    { 
        "agent_type": "predator",
        # "initial_coordinates": ((0.5,0.6),2*rd.random()*math.pi),
        "initial_coordinates": ((0.5,0.6),1.6),
        "radius": 20,
    },
    # { 
    #     "agent_type": "prey",
    #     # "initial_coordinates": ((0.1,0.9), 0.2),
    #     "initial_coordinates": ((0.5,0.4), -1.6),
    #     # "initial_coordinates": ((0.4,0.4), 2*rd.random()*math.pi),
    #     "radius": 15,
    # },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.5,0.4), 2*rd.random()*math.pi),
        "radius": 15,
    },
    # { 
    #     "agent_type": "prey",
    #     "initial_coordinates": ((0.6,0.4), 2*rd.random()*math.pi),
    #     "radius": 15,
    # },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.4,0.2), 2*rd.random()*math.pi),
        "radius": 15,
    },
    # { 
    #     "agent_type": "prey",
    #     "initial_coordinates": ((0.5,0.2), 2*rd.random()*math.pi),
    #     "radius": 15,
    # },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.6,0.2), 2*rd.random()*math.pi),
        "radius": 15,
    }
]

playground = {
    "size": (600,600),
    "interactions": ["BIG_EAT_SMALL"]
}

simulation = open_session(nb_agents=len(agents), agents=agents, playground_params=playground)
simulator = simulation[0]
predator = simulation[1]
preys = simulation[2:]

In [ ]:
simulator.status()

In [ ]:
# DEBUG CELL

for prey in preys:
    left, right = prey.prox_activations(tracked_objects=['Wall'])
    print(left, right)
    print(1- right, 1 - left)
    # compute_speed = lambda x: (math.tanh(-4*x+2)+1)/2
    # print(compute_speed(right), compute_speed(left))
    # print(1/100*(right-left))
    # print(1 - right - 1/(100*(right-left)), 1 - left + 1/(100*(right-left)))

# Testing aggression, fear and obstacle avoidance behaviors

In [3]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=['Wall'])
    return 1 - right, 1 - left
    
def aggression(robot):
    tracked_objects=["prey"]
    left, right = robot.prox_activations(tracked_objects)
    return right, left

def fear(robot):
    tracked_objects=["predator"]
    left, right = robot.prox_activations(tracked_objects)
    return left, right

In [6]:
for prey in preys:
    prey.detach_all_behaviors()
    prey.attach_behavior(obstacle_avoidance, freq=5)
    prey.attach_behavior(fear, freq=10)
    prey.start_behavior(fear)
    prey.start_behavior(obstacle_avoidance)

predator.detach_all_behaviors()
predator.attach_behavior(obstacle_avoidance, freq=5)
predator.attach_behavior(aggression, freq=10)
predator.start_behavior(aggression)
predator.start_behavior(obstacle_avoidance)
    

Behavior fear started
Behavior obstacle_avoidance started
Behavior fear started
Behavior obstacle_avoidance started
Behavior fear started
Behavior obstacle_avoidance started
Behavior aggression started
Behavior obstacle_avoidance started


In [5]:
for prey in preys:
    prey.stop_behavior(fear)
    prey.stop_behavior(obstacle_avoidance)
    prey.stop()

predator.stop_behavior(aggression)
predator.stop_behavior(obstacle_avoidance)
predator.stop()

Behavior fear stopped
Behavior obstacle_avoidance stopped
Behavior fear stopped
Behavior obstacle_avoidance stopped
Behavior fear stopped
Behavior obstacle_avoidance stopped
Behavior aggression stopped
Behavior obstacle_avoidance stopped


In [ ]:
close_session(simulator)

# Testing sphere apparition

In [176]:
simulator.start_sphere_apparition(period=10.)

Routine sphere_apparition started


In [177]:
def seek_spheres(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    left_wheel = 1 - left
    right_wheel = 1 - right
    return right, left

In [178]:
predator.detach_all_behaviors()

predator.attach_behavior(seek_spheres, freq=1)
predator.start_behavior(seek_spheres)

Behavior seek_spheres started


In [ ]:
simulator.stop_sphere_apparition()

In [ ]:
predator.stop_behavior(seek_spheres)
predator.stop()

In [ ]:
close_session(simulator)